<center> <h1>PREDICIÓN DEL SUEÑO CON MLP</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>


###### Indice

En este jupyter notebook se lleva a cabo la construcción de un clasificador MLP de salida binaria para predecir el el estado de vigilia o sueño ligero y  el de sueño profundo.

* **1. Cargar datos Train-Test**
  * **1.1 Preparación datos de entrada y salida**
* **2. Clasificador MLP binario**
   * **2.1 Creación del modelo y selección parámetros libres**
   * **2.3 Evaluación del modelo**

IMPORTAR MÓDULOS

In [1]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
import sklearn

## 1. CARGAR DATOS TRAIN-TEST

Se cargan los datos de train y test generados en los jupyter notebooks 'Nombre de los jupyter', en los que procesan los datos y se obtienen las características que se utilizan para entrenar los clasificadores. 

En el siguiente apartado se preparan los datos para seleccionar una salida binaria o multiclase según el clasificador que se vaya a utilizar.

### 2.1 Preparación datos de entrada y salida

In [2]:
#Train
Train= pd.read_csv('Train.csv').drop(['Unnamed: 0'], axis = 1)
Test= pd.read_csv('Test.csv').drop(['Unnamed: 0'], axis = 1)

In [3]:
X_train=Train.drop(['Etiquetas', 'Etiquetas binarias','ID_P'], axis = 1) 
y_train_bin=Train['Etiquetas binarias']
y_train_multi=Train['Etiquetas']

X_test=Test.drop(['Etiquetas', 'Etiquetas binarias','ID_P'], axis = 1)
y_test_bin=Test['Etiquetas binarias']
y_test_multi=Test['Etiquetas']

## 2. CLASIFICADOR MLP BINARIO

Se ha utilizado un perceptrón multicapa para clasificar la vigilia o sueño ligero del sueño profundo. Es decir, se ha construido un clasificador con salida binaria.

### 2.1  Creación del modelo y selección parámetros libres

Se proponen posibles valores de neuronas en la capa oculta para ver con cuál de ellos se obtienen las mejores prestaciones para el modelo. Se crea una tabla que recoge la pérdidas para cada valor del número de neuronas y aquel que tenga menos pérdida es el que se utilizará para el conjunto de test.

In [4]:
# Número de neuronas de la capa oculta
n_hidden_1=5
n_hidden_2=50
n_hidden_3=500
n_hidden_4=5000

n_hidden=[n_hidden_1, n_hidden_2, n_hidden_3, n_hidden_4]

In [5]:
# Tabla para recoger las prestaciones en train 
columns  = ['Loss','Score']
comp_train = pd.DataFrame(columns=columns)

for i in n_hidden:
    
    print('\033[4;32m' + 'MLP clasiffier with',i,'neurons in the hidden layer' + '\033[0m')

    model=MLPClassifier(activation='relu', batch_size=50, hidden_layer_sizes=(i,), learning_rate='constant',
        learning_rate_init=0.1, max_iter=30, shuffle=True, solver='sgd', tol=0.01, early_stopping=True,
        validation_fraction=0.2, verbose=True, n_iter_no_change=5)
    
    model.fit(X_train, y_train_bin)
    
    loss=model.loss_  
    comp_train = comp_train.set_value(i,('Loss','Score'),(loss,'Verbose'))

MLP clasiffier with 5 neurons in the hidden layer
Iteration 1, loss = inf
Validation score: 0.333333
Iteration 2, loss = 0.75514127
Validation score: 0.666667
Iteration 3, loss = 0.78321472
Validation score: 0.666667
Iteration 4, loss = 0.83302077
Validation score: 0.666667
Iteration 5, loss = 0.87608329
Validation score: 0.666667
Iteration 6, loss = 0.90725725
Validation score: 0.666667
Iteration 7, loss = 0.92595957
Validation score: 0.666667
Iteration 8, loss = 0.93824195
Validation score: 0.666667
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.
MLP clasiffier with 50 neurons in the hidden layer
Iteration 1, loss = inf
Validation score: 0.333333
Iteration 2, loss = 0.71268525
Validation score: 0.666667
Iteration 3, loss = inf
Validation score: 0.666667
Iteration 4, loss = 0.70763062
Validation score: 0.666667
Iteration 5, loss = 0.72175194
Validation score: 0.666667
Iteration 6, loss = 0.73293946
Validation score: 0.666667
Iteration 7, los

/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


Iteration 4, loss = 0.65944692
Validation score: 0.666667
Iteration 5, loss = 0.65336522
Validation score: 0.666667
Iteration 6, loss = 0.65616590
Validation score: 0.666667
Iteration 7, loss = 0.67067354
Validation score: 0.666667
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.


/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [6]:
comp_train

,Loss,Score
5,0.938242,Verbose
50,0.744023,Verbose
500,2.43057,Verbose
5000,0.670674,Verbose


Las medidas de prestación elegidas para la elección de una topología han sido el validation score y la pérdida. Ambas se pueden ver al aplicar el fit del modelo, gracias al parámetro verbose=True. La pérdida la hemos añadido a la siguiente tabla pero la accuracy de cada epoch para el conjunto de validación hay que verla en la celda anterior. Según estos resultados, la mejor topología de MLP sería la de **n_hidden_1 con 5 neuronas** en la capa oculta, ya que el par (loss,val_score) es el mejor de los 4.
En todos los casos el valor 'validation score' está entorno a 0.66, pero el número de neuronas de menor pérdida es 5.

¿Cómo sabíamos si estaba memorizando o no?

### 2.2 Evaluación del modelo

Testeamos el modelo con el valor de número de neuronas en la capa ocula que ofrece la menor pérdida (número neuronas = 5)

In [7]:
columns  = ['Accuracy','Recall','F1-score']
comp_test = pd.DataFrame(columns=columns)

print('\033[4;32m' + 'MLP clasiffier with',n_hidden_1,'neurons in the hidden layer' + '\033[0m')

model=MLPClassifier(activation='relu', batch_size=50, hidden_layer_sizes=(n_hidden_1), learning_rate='constant',
    learning_rate_init=0.1, max_iter=30, shuffle=True, solver='sgd', tol=0.01, early_stopping=True,
    validation_fraction=0.2, verbose=True, n_iter_no_change=5)

model.fit(X_train,y_train_bin)

predictions = model.predict(X_test)
cr=classification_report(y_test_bin,predictions,output_dict=True)
acc,rec,f1=cr['weighted avg']['precision'],cr['weighted avg']['recall'],cr['weighted avg']['f1-score']

comp_test = comp_test.set_value(n_hidden_1, ('Accuracy','Recall','F1-score'),(acc,rec,f1))

MLP clasiffier with 5 neurons in the hidden layer
Iteration 1, loss = inf
Validation score: 0.333333
Iteration 2, loss = 0.74452626
Validation score: 0.666667
Iteration 3, loss = 0.68019090
Validation score: 0.666667
Iteration 4, loss = 0.67571490
Validation score: 0.666667
Iteration 5, loss = 0.68773839
Validation score: 0.666667
Iteration 6, loss = 0.69694474
Validation score: 0.666667
Iteration 7, loss = 0.70092913
Validation score: 0.666667
Iteration 8, loss = 0.69969468
Validation score: 0.666667
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.


/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [8]:
comp_test

,Accuracy,Recall,F1-score
5,0.4489,0.67,0.537605


In [9]:
print("================Classification Report=================")
print(cr)

================Classification Report=================
{'0.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 33}, '1.0': {'precision': 0.67, 'recall': 1.0, 'f1-score': 0.8023952095808384, 'support': 67}, 'accuracy': 0.67, 'macro avg': {'precision': 0.335, 'recall': 0.5, 'f1-score': 0.4011976047904192, 'support': 100}, 'weighted avg': {'precision': 0.4489, 'recall': 0.67, 'f1-score': 0.5376047904191618, 'support': 100}}
